In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import surprise
from surprise import SVD, SVDpp, NormalPredictor
from sklearn.model_selection import GridSearchCV, cross_validate
from surprise import Dataset, Reader
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, KernelPCA
import category_encoders as ce
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('Data/Cleaned_Data/book_user_explicit_rating_cleaned.csv',encoding='UTF-8')
df_book = pd.read_csv('Data/Cleaned_Data/book_cleaned.csv',encoding='UTF-8')
df_rating = pd.read_csv('Data/Cleaned_Data/rating_cleaned.csv',encoding='UTF-8')
df_user = pd.read_csv('Data/Cleaned_Data/user_cleaned.csv',encoding='UTF-8')

In [3]:
df = df.drop(columns=['index'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df_book = df_book.loc[:, ~df_book.columns.str.contains('^Unnamed')]
df_rating = df_rating.loc[:, ~df_rating.columns.str.contains('^Unnamed')]
df_user = df_user.loc[:, ~df_user.columns.str.contains('^Unnamed')]
df = df.drop(columns=['Image_URL','ISBN'])

In [4]:
new_df = df.copy()
new_df_2 = df.copy()

In [5]:
cat_cols = ['Book_Title','Book_Author','Publisher','Country']

In [6]:
labelenc = LabelEncoder()
binenc = ce.BinaryEncoder(cols=cat_cols)

In [7]:
def label_encoding(df,param):
    df[param] = labelenc.fit_transform(df[param].values)
    df[param] = df[param].astype('category')

In [8]:
label_encoding(new_df, 'User_ID')
label_encoding(new_df, 'Unique_ISBN')
label_encoding(new_df, 'Book_Title')
label_encoding(new_df, 'Book_Author')
label_encoding(new_df, 'Publisher')
label_encoding(new_df, 'Country')
label_encoding(new_df, 'Age_Range')

In [9]:
new_df = binenc.fit_transform(new_df)

In [10]:
scaler = RobustScaler()

In [11]:
new_df_scaled = scaler.fit_transform(new_df)

In [12]:
kernel_pca = KernelPCA(
    n_components=2, kernel='rbf', 
                 gamma=15, random_state=42
)

df_kernel = kernel_pca.fit_transform(new_df_scaled)

: 

: 

# Paramter tuning..

In [ ]:
def my_scorer(estimator, X, y=None):
    X_reduced = estimator.transform(X)
    X_pre = estimator.inverse_transform(X_reduced)
    return -1 * mean_squared_error(X, X_pre)

In [ ]:
param_grid = [{
        "gamma": np.linspace(0.03, 0.05, 10),
        "kernel": ["rbf", "sigmoid", "linear", "poly"]
    }]
kpca=KernelPCA(fit_inverse_transform=True, n_jobs=-1) 


In [ ]:
grid_search = GridSearchCV(kpca, param_grid, cv=3, scoring=my_scorer)